## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd
import glob
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import requests
import time

## Data Read-in

In [2]:
csv_files = glob.glob('*.csv')

df_list = []

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    df_list.append(df)
    
df = pd.concat(df_list,ignore_index=True)

print(len(df))

2628


In [3]:
gdf = gpd.read_file('countyboundary')

## Previous month's values

Get values from here:

https://docs.google.com/spreadsheets/d/1GWlqTIzCB29pDVDjd-iBVejVGGsLiAd0P6fq4LM83WI/edit?usp=sharing

In [4]:
print('Input County Name:')
county_name = input()

Input County Name:
Miami-Dade


In [5]:
print('Input total condo units sold previous (not last) month:')
total_condos_sold_2nd_month = input()

Input total condo units sold previous (not last) month:
681


In [6]:
print('Input total sales volume ($$$ million) from previous (not last) month:')
sales_volume_2nd_month = input()

Input total sales volume ($$$ million) from previous (not last) month:
504000000


In [7]:
print('Input median sales price from previous (not last) month:')
median_sales_price_2nd_month = input()

Input median sales price from previous (not last) month:
390000


In [8]:
print('Input median psf from previous (not last) month:')
median_psf_2nd_month = input()

Input median psf from previous (not last) month:
395


## Data Clean

In [9]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [10]:
df = df.dropna(subset=['SOLD DATE'])

In [11]:
# Get the current date
current_date = datetime.now()

# Calculate the first day of the current month
first_day_of_current_month = current_date.replace(day=1)

# Get the last day of the previous month by subtracting one day from the first day of the current month
last_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

# Get the month name of the previous month
previous_month_name = last_day_of_previous_month.strftime("%B")

# ---- Get the name of the month, as a string, from two months ago.
# ---- Meaning, if the current month is January, return November.

# Get the first day of the previous month by subtracting one day from the first day of the current month
first_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

# Get the first day of the month before the last by subtracting one day from the first day of the previous month
first_day_of_month_before_last = first_day_of_previous_month.replace(day=1) - timedelta(days=1)

# Get the month name of the month before the last
month_before_last_name = first_day_of_month_before_last.strftime("%B")

print(f'Report month: {previous_month_name} ---- Month before report month: {month_before_last_name}')

Report month: November ---- Month before report month: October


In [12]:
# Define list of desired months (excluding current month)
desired_months = [previous_month_name]

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [13]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    608
Name: count, dtype: int64
-------
$/SQUARE FEET
False    603
True       5
Name: count, dtype: int64
-------
YEAR BUILT
False    608
Name: count, dtype: int64
-------


In [14]:
sorted_df = df_filtered.sort_values(by='YEAR BUILT', ascending=False)
second_newest_building = sorted_df.iloc[2]
print(second_newest_building['URL'])

https://www.redfin.com/FL/Miami/2655-S-Bayshore-Dr-33133/unit-1008/home/191916890


In [15]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [16]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [17]:
df_filtered = df_filtered.drop_duplicates().reset_index(drop=True)

In [18]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
360,PAST SALE,November-22-2024,Condo/Co-op,7985 SW 86th St #404,Miami,FL,33143.0,299.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/7985-SW-86th-S...,MARMLS,A11649943,N,Y,25.691237,-80.322984
341,PAST SALE,November-20-2024,Condo/Co-op,16500 Collins Ave #552,Sunny Isles Beach,FL,33160.0,885.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11669714,N,Y,25.928549,-80.125079
371,PAST SALE,November-5-2024,Condo/Co-op,20310 NE 2nd Ave Unit K16,Miami Gardens,FL,33179.0,65000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Gardens/20310-...,Beaches MLS,F10423878,N,Y,25.960654,-80.198883
379,PAST SALE,November-6-2024,Condo/Co-op,15610 NE 6th Ave Unit 24C,Miami,FL,33162.0,68000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/15610-NE-6th-A...,Beaches MLS,F10433775,N,Y,25.918155,-80.188500
376,PAST SALE,November-1-2024,Condo/Co-op,19201 Collins Ave #1043,Sunny Isles Beach,FL,33160.0,128000.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/19...,MARMLS,A11602936,N,Y,25.954318,-80.119900
377,PAST SALE,November-20-2024,Condo/Co-op,2851 Leonard Dr Unit J108,Aventura,FL,33160.0,130000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/2851-Leonar...,MARMLS,A11599985,N,Y,25.941099,-80.146740
382,PAST SALE,November-11-2024,Condo/Co-op,763 Pennsylvania Ave #116,Miami Beach,FL,33139.0,135000.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/763-Penn...,MARMLS,A11562923,N,Y,25.778229,-80.133794
380,PAST SALE,November-13-2024,Condo/Co-op,484 NW 165th St Rd Unit A615,Miami,FL,33169.0,135000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/484-NW-165th-S...,MARMLS,A11584600,N,Y,25.923659,-80.206675
369,PAST SALE,November-20-2024,Condo/Co-op,19201 Collins Ave #644,Sunny Isles Beach,FL,33160.0,149000.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/19...,MARMLS,A11625732,N,Y,25.954318,-80.119900
359,PAST SALE,November-12-2024,Condo/Co-op,1850 NE 169th St #212,North Miami Beach,FL,33162.0,150000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/18...,MARMLS,A11640140,N,Y,25.931140,-80.163317


In [21]:
print(df_filtered['URL'].iloc[341])

https://www.redfin.com/FL/Sunny-Isles-Beach/16500-Collins-Ave-33160/unit-552/home/43277267


In [23]:
# Correct the prices, if needed
df_filtered.at[360,'PRICE']=(299_000)
df_filtered.at[341,'PRICE']=(980_000)


# Correct the psf, if needed
df_filtered.at[360,'$/SQUARE FEET']=(299_000/833)
df_filtered.at[341,'$/SQUARE FEET']=(980_000/1604)

In [54]:
### If non-condo sales snuck in and needed to be dropped
# df_filtered = df_filtered.drop(index=107)

In [61]:
### If needed, drop the row with the lowest price
# min_value_index = df_filtered['PRICE'].idxmin()
# df_filtered = df_filtered.drop(min_value_index)

In [24]:
# print(df_filtered['URL'].iloc[528])

In [24]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
371,65000.0,20310 NE 2nd Ave Unit K16,Miami Gardens,84.0
379,68000.0,15610 NE 6th Ave Unit 24C,Miami,110.0
367,155000.0,1780 NE 191st St Unit 802-2,Miami,118.0
380,135000.0,484 NW 165th St Rd Unit A615,Miami,130.0
351,175000.0,1750 NE 191st St Unit 205-1,Miami,133.0
373,150000.0,20500 W Country Club Dr #308,Aventura,139.0
384,167000.0,922 NE 199th St Unit 1E,Miami,140.0
346,179900.0,671 NE 195th St Unit 220E,Miami,143.0
378,166000.0,12035 NE 2nd Ave Unit A120,North Miami,169.0
377,130000.0,2851 Leonard Dr Unit J108,Aventura,170.0


In [63]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [25]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [26]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [27]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [28]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [29]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [30]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"{previous_month_name} 2024 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

folium.GeoJson(gdf,tooltip=f'{county_name}',name=f'{county_name}').add_to(m)

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
# m

In [31]:
m.save('index.html')

## Summary Info

In [32]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [33]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [34]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [35]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami-Beach/1320-Drexel-Ave-33139/unit-306/home/42796415


In [36]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
BAYVIEW FISHER ISL CONDO, 5211 Fisher Island Dr #5211 Miami Beach | Price $11,350,000 | $2,830 psf | Year built: 1992
Least Expensive
Romont South, 20310 NE 2nd Ave Unit K16 Miami Gardens | Price $65,000 | $84 psf | Year built: 1972
Highest Price Per Square Foot
OCEANA BAL HARBOUR CONDO, 10203 Collins Ave #1702 Bal Harbour | Price $10,875,000 | $3,028 psf | Year built: 2016
Lowest Price Per Square Foot
Romont South, 20310 NE 2nd Ave Unit K16 Miami Gardens | Price $65,000 | $84 psf | Year built: 1972
Newest
FIVE PARK, 500 Alton #4402 Miami Beach | Price $8,712,500 | $2,974 psf | Year built: 2024
Oldest
WINDSOR PLAZA CONDO, 1320 Drexel Ave #306 Miami Beach | Price $318,000 | $548 psf | Year built: 1925


## Map URL Snagger

In [37]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDadeCounty_condo_sales_month_ending_nov_2024


## Get Summary Data

In [40]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 568
--------
Total sale price: $425,584,505
--------
Median sale price: $415,000
--------
Max sale price: $11,350,000
--------
Min sale price: $65,000
------------------------------------------------
PSF INFO
Max price per square foot: $3,028
--------
Min price per square foot: $84
--------
Median price per square foot: $430
------------------------------------------------
CONDO AGES
Newest building: 2024.0
----------
Oldest building: 1925.0
----------
Average building age: 1988.9014084507041


## Collect Agent Data

In [41]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [42]:
df_filtered.reset_index(inplace=True,drop=True)

In [43]:
max_price_index = df_filtered['PRICE'].idxmax()
min_price_index = df_filtered['PRICE'].idxmin()

max_price_psf_index = df_filtered['$/SQUARE FEET'].idxmax()
min_price_psf_index = df_filtered['$/SQUARE FEET'].idxmin()

max_year_built_index = df_filtered['YEAR BUILT'].idxmax()
min_year_built_index = df_filtered['YEAR BUILT'].idxmin()

In [44]:
index_list = [max_price_index,min_price_index,
             max_price_psf_index,min_price_psf_index,
             max_year_built_index,min_year_built_index]

In [45]:
response_list = []

for index, row in df_filtered.iterrows():
    if index in index_list:
        response_list.append(process_response(row['URL']))
        
response_df = pd.DataFrame(response_list)

merged_df = pd.merge(left=df_filtered,left_on='URL',right=response_df, right_on='url',how='outer')

In [46]:
### Test to see if data was collected
merged_df['list_agent_name_1'].unique()

array([nan, 'Ethan Assouline', 'Matthieu-Rez Mamoudi',
       'Christopher Wands', 'Julian Johnston', 'Brett Matthews'],
      dtype=object)

In [64]:
df_filtered = merged_df

In [65]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_broker_name_2']}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_broker_name_2']}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_broker_name_2']}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_broker_name_2']}")

Most Expensive
BAYVIEW FISHER ISL CONDO, 5211 Fisher Island Dr #5211 Miami Beach | Price $11,350,000 | $2,830 psf | Year built: 1992 | Listing agents: Julian Johnston with The Corcoran Group and Richard Taylor with The Corcoran Group | Buyer's agents: None with None and None with None
Least Expensive
Romont South, 20310 NE 2nd Ave Unit K16 Miami Gardens | Price $65,000 | $84 psf | Year built: 1972 | Listing agents: Brett Matthews with I Deal Real Estate Inc and None with None | Buyer's agents: Fabiana Navas with Clover 4 Group LLC and None with None
Highest Price Per Square Foot
OCEANA BAL HARBOUR CONDO, 10203 Collins Ave #1702 Bal Harbour | Price $10,875,000 | $3,028 psf | Year built: 2016 | Listing agents: Ethan Assouline with Douglas Elliman and Ruthie Assouline with Douglas Elliman | Buyer's agents: Anna Sherrill with One Sotheby's International Re and Adriano Biasella with One Sotheby's International Realty
Lowest Price Per Square Foot
Romont South, 20310 NE 2nd Ave Unit K16 Miami

## Get condo names and locations for most and least expensive

In [66]:
most_expensive_muni_condo_name = df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']
most_expensive_muni_location = df_filtered.loc[df_filtered['PRICE'].idxmax()]['CITY']

least_expensive_muni_condo_name = df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']
least_expensive_muni_location = df_filtered.loc[df_filtered['PRICE'].idxmin()]['CITY']

In [67]:
## Calculate the increase/descrease between both months
subject_month_sales_volume = df_filtered["PRICE"].sum()
prior_month_sales_volume = int(sales_volume_2nd_month)  # Example value, replace with actual value

subject_month_closings = len(df_filtered)
prior_month_closings = int(total_condos_sold_2nd_month)

# Subtract the smaller value from the larger one
if subject_month_sales_volume > prior_month_sales_volume:
    hed_rf = 'jumps'
    nut_graf = 'rising'
    second_nut_graf_ref = 'up'
    social = 'higher'
    seo = 'increased'
elif prior_month_sales_volume > subject_month_sales_volume:
    hed_rf = 'drops'
    nut_graf = 'falling'
    second_nut_graf_ref = 'down'
    social = 'lower'
    seo = 'dropped'
else:
    hed_rf = 'equals'
    nut_graf = 'equaling'
    second_nut_graf_ref = 'on par with'
    social = 'equal'
    seo = "did't change"
    
    
if subject_month_closings > prior_month_closings:
    dek_rf = 'rose'
elif prior_month_closings > subject_month_closings:
    dek_rf ='fell'
else:
    dek_rf = 'equaled'

In [68]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK', 'FULL_ADDRESS', 'url',
       'list_agent_name_1', 'list_broker_name_1', 'list_agent_name_2',
       'list_broker_name_2', 'buy_agent_name_1', 'buy_broker_name_1',
       'buy_agent_name_2', 'buy_broker_name_2'],
      dtype='object')

In [69]:
# Get values for the top sale (max price)
top_sale_listing_agent_1 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'list_agent_name_1']
top_sale_listing_broker_1 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'list_broker_name_1']
top_sale_listing_agent_2 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'list_agent_name_2']
top_sale_listing_broker_2 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'list_broker_name_2']
top_sale_buyer_agent_1 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'buy_agent_name_1']
top_sale_buyer_broker_1 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'buy_broker_name_1']
top_sale_buyer_agent_2 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'buy_agent_name_2']
top_sale_buyer_broker_2 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'buy_broker_name_2']

# Get values for the bottom sale (min price)
bottom_sale_listing_agent_1 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'list_agent_name_1']
bottom_sale_listing_broker_1 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'list_broker_name_1']
bottom_sale_listing_agent_2 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'list_agent_name_2']
bottom_sale_listing_broker_2 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'list_broker_name_2']
bottom_sale_buyer_agent_1 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'buy_agent_name_1']
bottom_sale_buyer_broker_1 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'buy_broker_name_1']
bottom_sale_buyer_agent_2 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'buy_agent_name_2']
bottom_sale_buyer_broker_2 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'buy_broker_name_2']

# Now create the formatted string
test_string = f'''


TOP SALE LISTING AGENTS:

{top_sale_listing_agent_1} with {top_sale_listing_broker_1} and {top_sale_listing_agent_2} with {top_sale_listing_broker_2}

TOP SALE BUYER AGENTS:

{top_sale_buyer_agent_1} with {top_sale_buyer_broker_1} and {top_sale_buyer_agent_2} with {top_sale_buyer_broker_2}

BOTTOM SALE LISTING AGENTS:

{bottom_sale_listing_agent_1} with {bottom_sale_listing_broker_1} and {bottom_sale_listing_agent_2} with {bottom_sale_listing_broker_2}

BOTTOM SALE BUYER AGENTS:

{bottom_sale_buyer_agent_1} with {bottom_sale_buyer_broker_1} and {bottom_sale_buyer_agent_2} with {bottom_sale_buyer_broker_2}

'''

print(test_string)





TOP SALE LISTING AGENTS:

Julian Johnston with The Corcoran Group and Richard Taylor with The Corcoran Group

TOP SALE BUYER AGENTS:

None with None and None with None

BOTTOM SALE LISTING AGENTS:

Brett Matthews with I Deal Real Estate Inc and None with None

BOTTOM SALE BUYER AGENTS:

Fabiana Navas with Clover 4 Group LLC and None with None




In [70]:
story_string = f'''
\033[1mHED:\033[0m {county_name} condo sales dollar volume {hed_rf} in {previous_month_name} to ${round(df_filtered["PRICE"].sum()/1_000_000)}M 
\033[1mDEK:\033[0m Number of closings {dek_rf} to {len(df_filtered)} from {total_condos_sold_2nd_month} in {month_before_last_name}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county_name} {previous_month_name} Condo Sales Report 
\033[1mSEO DESCRIPTION:\033[0m {county_name}’s condo sales volume {seo} to ${round(df_filtered["PRICE"].sum()/1_000_000)} million in {previous_month_name}.
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m  
\033[1mSocial:\033[0m {county_name} had ${round(df_filtered["PRICE"].sum()/1_000_000):,.0f} million in condo sales in {previous_month_name}, {social} than ${round(int(sales_volume_2nd_month)/1_000_000)} million from {month_before_last_name}, @afarence reports 
\033[1mART:\033[0m 

CHANGE ME

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county_name}, monthly condo sales, condos, condo market, {most_expensive_muni_condo_name.title()}, {most_expensive_muni_location}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m

{county_name}’s NEWS PEG HERE.

{previous_month_name} condo sales totaled ${round(df_filtered["PRICE"].sum()/1_000_000)} million, {nut_graf} from ${int(sales_volume_2nd_month)/1_000_000:.0f} million in {month_before_last_name}. Brokers closed {len(df_filtered)} sales last month, {second_nut_graf_ref} from {total_condos_sold_2nd_month} sales in {month_before_last_name} and XXX in XXX, Multiple Listing Service data from Redfin show.

{previous_month_name} sale prices ranged from ${df_filtered["PRICE"].min():,.0f} to ${df_filtered["PRICE"].max():,.0f} million, with a median sale price of ${df_filtered["PRICE"].median():,.0f}. The price per square foot ranged from ${df_filtered["$/SQUARE FEET"].min():,.0f} to ${df_filtered["$/SQUARE FEET"].max():,.0f}, with a median of ${df_filtered["$/SQUARE FEET"].median():,.0f} per square foot.

In {month_before_last_name}, sales closed with a median price of ${int(median_sales_price_2nd_month):,.0f}, and ${median_psf_2nd_month} per square foot.

A ${df_filtered["PRICE"].max():,.0f} million closing at {most_expensive_muni_condo_name.title()} took the title of priciest sale last month. {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} sold for ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} per square foot after XX days on the market. XXX with XXX had the listing, and XXX with XXX represented the buyer.

TOP SALE LISTING AGENTS:

{top_sale_listing_agent_1} with {top_sale_listing_broker_1} and {top_sale_listing_agent_2} with {top_sale_listing_broker_2}

TOP SALE BUYER AGENTS:

{top_sale_buyer_agent_1} with {top_sale_buyer_broker_1} and {top_sale_buyer_agent_2} with {top_sale_buyer_broker_2}

The sale price was \033[1mmore/less\033[0m than {month_before_last_name}'s priciest sale. Unit XXXX 

In contrast, {previous_month_name}'s cheapest sale was an {least_expensive_muni_condo_name} in {least_expensive_muni_location}, at {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']}. Unit XXX traded for ${df_filtered['PRICE'].min():,.0f} — or ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} per square foot — after XXX days on the market. XXX with XXX handled both sides of the deal.

BOTTOM SALE LISTING AGENTS:

{bottom_sale_listing_agent_1} with {bottom_sale_listing_broker_1} and {bottom_sale_listing_agent_2} with {bottom_sale_listing_broker_2}

BOTTOM SALE BUYER AGENTS:

{bottom_sale_buyer_agent_1} with {bottom_sale_buyer_broker_1} and {bottom_sale_buyer_agent_2} with {bottom_sale_buyer_broker_2}

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="800" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of {previous_month_name}’s notable condo sales:
'''

In [71]:
print(story_string)

print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_broker_name_2']}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_broker_name_2']}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_broker_name_2']}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_broker_name_2']}")


HED: Miami-Dade condo sales dollar volume drops in November to $426M 
DEK: Number of closings fell to 568 from 681 in October
FEATURED HED:
SEO HED: Miami-Dade November Condo Sales Report 
SEO DESCRIPTION: Miami-Dade’s condo sales volume dropped to $426 million in November.
AUTHOR: Adam Farence
RESEARCH:  
Social: Miami-Dade had $426 million in condo sales in November, lower than $504 million from October, @afarence reports 
ART: 

CHANGE ME

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, monthly condo sales, condos, condo market, Bayview Fisher Isl Condo, Miami Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:

Miami-Dade’s NEWS PEG HERE.

November condo sales totaled $426 million, falling from $504 million in October. Brokers closed 568 sales last month, down from 681 sales in October and XXX in XXX, Multiple Listing Service data from Redfin s

# Print links for notable sales

## Top sale

In [72]:
print(df_filtered.loc[df_filtered['PRICE'].idxmax()]['URL'])

https://www.redfin.com/FL/Miami-Beach/5211-Fisher-Island-Dr-33109/unit-5211/home/43102905


## Cheapest sale

In [73]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami-Gardens/20310-NE-2nd-Ave-33179/unit-K16/home/189732641


## Highest PSF

In [ ]:
# df_filtered = df_filtered.drop(index=175)

In [74]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['URL'])

https://www.redfin.com/FL/Bal-Harbour/10203-Collins-Ave-33154/unit-1702/home/56745853


## Lowest PSF

In [75]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami-Gardens/20310-NE-2nd-Ave-33179/unit-K16/home/189732641


## Newest

In [76]:
df_filtered['YEAR BUILT'].idxmax()

180

In [78]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['URL'])

https://www.redfin.com/FL/Miami-Beach/500-Alton-Rd-33139/unit-4402/home/190593534


## Oldest

In [79]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami-Beach/1320-Drexel-Ave-33139/unit-306/home/42796415


## Time on Market Calculator

In [81]:
from datetime import datetime, timedelta

################ YEAR, MONTH, DAY #######################

date1 = datetime(2023, 12, 14) ## List (Earlier) date
date2 = datetime(2024, 11, 5) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

327


## Access second highest (or whatever is needed)

In [58]:
# ### Second highest ###
# # Sort the DataFrame by '$/SQUARE FEET' in descending order
# second_highest = df_filtered.sort_values(by='$/SQUARE FEET', ascending=False)

# # Get the URL of the entry with the second highest '$/SQUARE FEET' value
# second_highest_url = second_highest.iloc[1]['URL']  # `.iloc[1]` accesses the second row

# print(second_highest_url)